In [10]:
import numpy as np
import pandas as pd
import seaborn as sns

In [219]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

In [220]:
# Forward Stagewise Regression

def get_residuals(model, x, y):
    return y - model.predict(x)

def find_max_correlation(res, x, verbose = False):
    im, corr = 0, 0
    for i in range(x.shape[1]):
        cf = np.corrcoef(x[:,i], res)[0,1]
        if np.abs(cf) > np.abs(corr): im, corr = i, cf
        if verbose: print ("[+] %d: %f" % (im, cf)) 
    return im, corr

def get_coeff(x,y):
    return np.dot(x,y) / np.dot(x,x)

def update_model(model, idx, value):
    model.coef_[idx]+=value

def stagewise_regression(x, y, tolerance = 1e-4, max_iterations = 1e3, verbose = 0):
    model = LinearRegression()
    model.coef_ = np.zeros(x.shape[1])
    model.intercept_ = np.mean(y, axis = 0)
    
    it, corr = 0, tolerance * 2
    while abs(corr) > tolerance: 
        it+=1
        res = get_residuals(model, x, y)
        ix, corr = find_max_correlation(res, x)
        cf = get_coeff(x[:,ix], res)
        if cf == 0: 
            print("[!!] Coefficient not being updated")
            break
        update_model(model, ix, cf)
        if verbose == 2: 
            print("[+] Residuals: %f. Max corr: %f in cord %d, coeff: %f" % (np.dot(res, res), corr, ix, cf))
        if it > max_iterations: 
            print("[!!] Max iterations")
            break
    if verbose == 1:
        print("[+] Residuals: %f. Max corr: %f in cord %d, coeff: %f" % (np.dot(res, res), corr, ix, cf))
    return model
    
    

In [231]:
# Lets use one of the sklearn datasets
a = load_diabetes()
x = a['data']
y = a['target']
m = stagewise_regression(x, y, tolerance = 1e-5, max_iterations = 1e4, verbose = 1)

[+] Residuals: 1263983.184158. Max corr: 0.000010 in cord 8, coeff: 0.010770


In [232]:
# Let's compare the coefficients with a regular Linear regression
lr = LinearRegression()
lr.fit(x,y)
res = lr.predict(x) - y
np.dot(res,res)

1263983.156255485

In [233]:
(lr.coef_ - m.coef_)/ lr.coef_ * 100

array([ 0.11668894,  0.0043359 , -0.00482389,  0.00622631,  0.16086764,
        0.20895246,  0.57344995,  0.10845026,  0.06192832, -0.01649127])